# 營收統計

In [57]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import date

def 連接資料庫(db):

    # 連接到資料庫【月】 =================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    sql = '''select * from revenue t'''
    dfRevenue = pd.read_sql(sql,conn)

    # 連接到資料庫【季】
    sql2 = '''select * from financialStatements t'''
    dfFin = pd.read_sql(sql2,conn)

    # --------------------------------------
    # 連接到資料庫【年】
    conn2 = sqlite3.connect('guli.db')
    sql3 = '''select * from guli t'''
    df_guli = pd.read_sql(sql3,conn2) 
    # --------------------------------------

    # 連接到資料庫【上市上櫃】
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    ids = dfStock['code']
    # ==================================================================================
    conn.close()
    
    return dfRevenue,dfFin,df_guli,ids


def 月營收統計(dfRevenue, stocks):

    # 整理【月營收】欄位
    dfRevenue2 = dfRevenue.copy()
    dfRevenue2 = dfRevenue2.drop(columns=['open','high','low','updownYuan','updown'])
    dfRevenue2.columns = ['代碼','月別','收盤','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)']

    # 【月營收資料】
    dataExport1 = pd.DataFrame(columns = ['代碼','收排','收盤','營收(億)','月增(%)','年增(%)','月別','年前3','累計營收(億)','累計年增(%)','當月營收排名','累計排名','營排'])


    for id in stocks:
        dfRevenue2mask = dfRevenue2['代碼'] == id
        dfRevenue3 = dfRevenue2[dfRevenue2mask]
        dfRevenue3 = dfRevenue3.sort_values(by='月別', ascending=False) # 重整月份
        dfRevenue3 = dfRevenue3[:60]

        # 收盤排名,當月營收排名 ========================================================
        dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
        dfRevenue3.insert(9,'當月營收排名',dfRevenue3['營收(億)'].rank(ascending=False))

        # 當月累計排名 ================================================================
        當月 = dfRevenue3.iloc[0]['月別'].split('/',1) # 去除年份取月份
        當月 = 當月[1]
        當月 = "/" + 當月
        累計 = dfRevenue3['月別'].str.contains(當月,na=False) # 取出所有同期月份，並為同期月份進行排名
        dfRevenue3.insert(10,'累計排名',dfRevenue3[累計]['累計營收(億)'].rank(ascending=False))
        dfRevenue3 = dfRevenue3.fillna(0)
        # 年前3 ======================================================================
        總年數 = []
        for k in range(len(dfRevenue3['月別'])):
            當年 = dfRevenue3.iloc[k]['月別'].split('/',1)
            當年 = 當年[0]
            if 當年 not in 總年數:
                總年數.append(當年)
        排3 = pd.DataFrame(columns = ['代碼','月別','收盤','營收(億)','月增(%)','年增(%)','累計營收(億)','累計年增(%)','收排','當月營收排名','累計排名','營排'])
        for e in range(len(總年數)):
            當年 = 總年數[e]
            前3年 = dfRevenue3['月別'].str.contains(當年,na=False) 
            dfRevenue4 = dfRevenue3[前3年].copy()
            dfRevenue4['營排'] = dfRevenue4['營收(億)'].rank(ascending=False)
            排3 = pd.concat([排3,dfRevenue4],axis=0,ignore_index = True)
        
        # ============================================================================
        
        
        dataExport1 = pd.concat([dataExport1,排3],axis=0,ignore_index = True)
    dataExport1a = dataExport1.loc[:,['代碼','收排','收盤','月增(%)','年增(%)','月別','營排','累計年增(%)','當月營收排名','累計排名']]
    
    return dataExport1a


def 季財報統計(dfFin, stocks):    
    dfFin2 = dfFin.copy()
    # loc可以使用文字，iloc則是使用數字
    dfFin2 = dfFin2.loc[:,['代碼','營業利益率','每股稅後盈餘(元)','稅後淨利年成長率','營業利益年成長率','季度']]

    # 【季財報資料】
    dataExport2 = pd.DataFrame(columns = ['代碼','營業利益率','盈比','盈4','盈單','稅後淨利年成長率','營業利益年成長率','季度','每股稅後盈餘(元)'])

    for id in stocks:
        dfFin2mask = dfFin2['代碼'] == id
        dfFin3 = dfFin2[dfFin2mask]
        dfFin3 = dfFin3.sort_values(by='季度', ascending=False) # 重整排名

        盈餘單 = []
        盈餘4 = []

        for y in range(len(dfFin3['季度'])):
            期別期 = dfFin3.iloc[y]['季度'][4:]

            if 期別期 == 'Q1':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+1]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q2':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+2]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q3':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+3]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q4':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            
        try:
            dfFin3['盈單'] = 盈餘單
        except:
            dfFin3['盈單'] = 0

        try:
            dfFin3['盈4'] = 盈餘4
        except:
            dfFin3['盈4'] = 0

        dfFin3['盈比'] = dfFin3['盈單']*4/dfFin3['盈4']
        dfFin3.replace([np.inf, -np.inf], 0, inplace=True)
        dataExport2 = pd.concat([dataExport2,dfFin3],axis=0,ignore_index = True)
    dataExport2a = dataExport2.iloc[:,:8]
    return dataExport2a


def 整理股利資料(df_guli):
    thisyear = date.today().year
    lastyear = thisyear-1
    前三年 = thisyear-2
    前四年 = thisyear-3
    前五年 = thisyear-4

    # 【今年】=======================================================================================================================================
    # 筛选出日期为今年的股利数据
    df_guli_thisyear = df_guli.copy()
    df_guli_thisyear['除息交易日'] = pd.to_datetime(df_guli_thisyear['除息交易日'])
    df_guli_thisyear = df_guli_thisyear[df_guli_thisyear['除息交易日'].dt.year == thisyear]

    # 进行分组和聚合操作，以计算今年每只股票的股利合计
    df_guli_thisyear_combined = df_guli_thisyear.groupby(['stock_id', df_guli_thisyear['除息交易日'].dt.year]).agg({'股利合計': 'sum'}).reset_index()
    # ==============================================================================================================================================
    
    
    # 【去年】=======================================================================================================================================
    # 筛选出日期为去年的股利数据
    df_guli_lastyear = df_guli.copy()
    df_guli_lastyear['除息交易日'] = pd.to_datetime(df_guli_lastyear['除息交易日'])
    df_guli_lastyear = df_guli_lastyear[df_guli_lastyear['除息交易日'].dt.year == lastyear]

    # 进行分组和聚合操作，以计算去年每只股票的股利合计
    df_guli_lastyear_combined = df_guli_lastyear.groupby(['stock_id', df_guli_lastyear['除息交易日'].dt.year]).agg({'股利合計': 'sum'}).reset_index()
    # ==============================================================================================================================================
    
    
    # 前三年=======================================================================================================================================
    # 筛选出日期为去年的股利数据
    df_guli_前三年 = df_guli.copy()
    df_guli_前三年['除息交易日'] = pd.to_datetime(df_guli_前三年['除息交易日'])
    df_guli_前三年 = df_guli_前三年[df_guli_前三年['除息交易日'].dt.year == 前三年]

    # 进行分组和聚合操作，以计算去年每只股票的股利合计
    df_guli_前三年_combined = df_guli_前三年.groupby(['stock_id', df_guli_前三年['除息交易日'].dt.year]).agg({'股利合計': 'sum'}).reset_index()
    # ==============================================================================================================================================
    
    
    # 前四年=======================================================================================================================================
    # 筛选出日期为去年的股利数据
    df_guli_前四年 = df_guli.copy()
    df_guli_前四年['除息交易日'] = pd.to_datetime(df_guli_前四年['除息交易日'])
    df_guli_前四年 = df_guli_前四年[df_guli_前四年['除息交易日'].dt.year == 前四年]

    # 进行分组和聚合操作，以计算去年每只股票的股利合计
    df_guli_前四年_combined = df_guli_前四年.groupby(['stock_id', df_guli_前四年['除息交易日'].dt.year]).agg({'股利合計': 'sum'}).reset_index()
    # ==============================================================================================================================================
    
    
    # 前五年=======================================================================================================================================
    # 筛选出日期为去年的股利数据
    df_guli_前五年 = df_guli.copy()
    df_guli_前五年['除息交易日'] = pd.to_datetime(df_guli_前五年['除息交易日'])
    df_guli_前五年 = df_guli_前五年[df_guli_前五年['除息交易日'].dt.year == 前五年]

    # 进行分组和聚合操作，以计算去年每只股票的股利合计
    df_guli_前五年_combined = df_guli_前五年.groupby(['stock_id', df_guli_前五年['除息交易日'].dt.year]).agg({'股利合計': 'sum'}).reset_index()
    # ==============================================================================================================================================
    
    return df_guli_thisyear_combined, df_guli_lastyear_combined, df_guli_前三年_combined, df_guli_前四年_combined, df_guli_前五年_combined


def 年營收統計(dataExport2a, df_combined, stocks):
    # 【年營收資料】
    dataExport3 = pd.DataFrame(columns= ['代碼', '股利發放年度', '股利合計', '盈4', '盈配率'])
    
    for id in stocks:
        
        # 月報表
        dataExport2a股息年 = dataExport2a[dataExport2a['代碼'] == id]
        
        # 股利
        df_combined_filter = df_combined[df_combined['stock_id'] == id]
        
        
        # 整理盈4 ============================================================================================================
        dataExport2a股息年_filtered = dataExport2a股息年[['代碼', '盈4', '季度']].iloc[1:] # 篩選出了最新一筆資料以外的內容
        dataExport2a股息年_filtered = dataExport2a股息年_filtered[dataExport2a股息年_filtered['季度'].str.endswith('Q4')] # 只提取第四季的內容（盈4）
        dataExport2a股息年_new = dataExport2a股息年[['代碼', '盈4', '季度']].iloc[:1] # 篩選出最新一筆資料的內容
        dataExport2a股息年2 = pd.concat([dataExport2a股息年_new, dataExport2a股息年_filtered], ignore_index=True) # 合併
        dataExport2a股息年2['季度'] = dataExport2a股息年2['季度'].str.replace('Q.*$', '', regex=True) # 刪除“Q”以及“Q”以後得內容

        dataExport2a股息年2.columns = ['stock_id', '盈4', 'year'] # 更改名字
        dataExport2a股息年2['year'] = pd.to_datetime(dataExport2a股息年2['year']) # str轉換日期
        # 提取年份并创建新的列
        dataExport2a股息年2['date'] = dataExport2a股息年2['year'].dt.year

        # 删除原始的year列（可选）
        dataExport2a股息年2 = dataExport2a股息年2.drop('year', axis=1)
        # ==================================================================================================================
        
        # 合併盈4和計算盈配率
        merged_df = pd.merge(df_combined_filter, dataExport2a股息年2, on=['stock_id', 'date'], how='left')
        # 计算盈配率
        merged_df['盈配率'] = merged_df['股利合計'] / merged_df['盈4']
        # 处理没有股利的年份，将盈配率设为0
        merged_df.loc[merged_df['股利合計'].isnull(), '盈配率'] = 0
        merged_df.fillna(0)
        merged_df.columns = ['代碼', '股利發放年度', '股利合計', '盈4', '盈配率']
        
        dataExport3 = pd.concat([dataExport3,merged_df],axis=0,ignore_index = True)
        
    return dataExport3
    


def 導出月季年(stockMonth,stockSeason,stockYear):
    Today = pd.Timestamp.today().strftime("%Y%m") # 格式化日期

    stockMonth.to_csv(f"01月營收統計{Today}.csv", encoding="big5hkscs",index=False)
    stockSeason.to_csv(f"02季營收統計{Today}.csv", encoding="big5hkscs",index=False)
    stockYear.to_csv(f"03年營收統計{Today}.csv", encoding="big5hkscs",index=False)
    
    
def 營收統計():
    dfRevenue,dfFin,df_guli,ids = 連接資料庫('goodinfoRevenue.db')
    dataExport1a = 月營收統計(dfRevenue, ids)
    dataExport2a = 季財報統計(dfFin, ids)
    
    # 年營收
    df_guli_thisyear_combined, df_guli_lastyear_combined, df_guli_前三年_combined, df_guli_前四年_combined, df_guli_前五年_combined = 整理股利資料(df_guli)
    # df_combined為5年全部已公佈的股利
    df_combined = pd.concat([df_guli_thisyear_combined, df_guli_lastyear_combined, df_guli_前三年_combined, df_guli_前四年_combined, df_guli_前五年_combined], ignore_index=True)
    df_combined.columns = ['stock_id', 'date', '股利合計']
    
    dataExport3 = 年營收統計(dataExport2a, df_combined, ids)
    
    導出月季年(dataExport1a, dataExport1a, dataExport3)


if __name__ == "__main__":
    營收統計()
    print('已導出全部資料')

C:\Users\PAN\AppData\Local\Temp\ipykernel_10160\719563278.py:53: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_10160\719563278.py:53: FutureWarning: Calling Series.rank with numeric_only=True and dtype object is deprecated and will raise a TypeError in a future version of pandas
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_10160\719563278.py:53: FutureWarning: Dropping of nuisance columns in Series.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  dfRevenue3.insert(8,'收排',dfRevenue3['收盤'].rank(ascending=False))
C:\Users\PAN\AppData\Local\Temp\ipykernel_10160\719563278.py:53: FutureWarning: Calling Series.rank with numeric_only=True

已導出全部資料
